In [1]:
%load_ext autoreload
%autoreload 2
"""https://adventofcode.com/2024/day/9"""

from aoc_utils import fetch_input_data
import logging
logging.basicConfig(level=logging.INFO)
# logging.basicConfig(level=logging.DEBUG)
from itertools import product

logger = logging.getLogger(__name__)
actual_input = fetch_input_data(2024, 9)

In [2]:
def parse_disk_map(disk_map):
    """Convert disk map string to list of alternating file/space lengths."""
    return [int(c) for c in disk_map]

def create_initial_disk(disk_map):
    """Create initial disk state with file IDs and free spaces."""
    lengths = parse_disk_map(disk_map)
    disk = []
    file_id = 0
    
    for i in range(0, len(lengths), 2):
        # Add file blocks
        file_length = lengths[i]
        disk.extend([file_id] * file_length)
        file_id += 1
        
        # Add free space blocks if not at end
        if i + 1 < len(lengths):
            space_length = lengths[i + 1]
            disk.extend(['.'] * space_length)
    
    return disk

def find_leftmost_space(disk):
    """Find the leftmost free space position in the disk."""
    for i, block in enumerate(disk):
        if block == '.':
            return i
    return None

def find_rightmost_file(disk):
    """Find the rightmost file block position in the disk."""
    for i in range(len(disk) - 1, -1, -1):
        if disk[i] != '.':
            return i
    return None

def compact_disk(disk):
    """Compact the disk by moving files left to fill gaps."""
    steps = [disk.copy()]
    
    while True:
        space_pos = find_leftmost_space(disk)
        if space_pos is None:
            break
            
        file_pos = find_rightmost_file(disk[space_pos:])
        if file_pos is None:
            break
            
        # Convert file_pos to absolute position
        file_pos += space_pos
        
        # Move the file block
        disk[space_pos] = disk[file_pos]
        disk[file_pos] = '.'
        
        steps.append(disk.copy())
    
    return steps

def calculate_checksum(disk):
    """Calculate filesystem checksum."""
    checksum = 0
    for pos, block in enumerate(disk):
        if block != '.':
            checksum += pos * block
    return checksum

def solve_disk_fragmenter(disk_map):
    """Solve the disk fragmenter puzzle."""
    # Create initial disk state
    disk = create_initial_disk(disk_map)
    
    # Compact the disk
    final_state = compact_disk(disk)[-1]
    
    # Calculate checksum
    return calculate_checksum(final_state)

# Test with examples
example1 = "12345"
example2 = "2333133121414131402"

# Print initial and final states for example1
disk1 = create_initial_disk(example1)
steps1 = compact_disk(disk1)
print("\nExample 1:")
print("Initial:", ''.join(str(x) for x in steps1[0]))
print("Final:", ''.join(str(x) for x in steps1[-1]))

# Print initial and final states for example2
disk2 = create_initial_disk(example2)
steps2 = compact_disk(disk2)
print("\nExample 2:")
print("Initial:", ''.join(str(x) for x in steps2[0]))
print("Final:", ''.join(str(x) for x in steps2[-1]))
print("Checksum:", calculate_checksum(steps2[-1]))  # Should be 1928


Example 1:
Initial: 0..111....22222
Final: 022111222......

Example 2:
Initial: 00...111...2...333.44.5555.6666.777.888899
Final: 0099811188827773336446555566..............
Checksum: 1928


In [ ]:
# Print initial and final states for example2
disk2 = create_initial_disk(example2)
steps2 = compact_disk(disk2)
print("\nExample 2:")
print("Initial:", ''.join(str(x) for x in steps2[0]))
print("Final:", ''.join(str(x) for x in steps2[-1]))
print("Checksum:", calculate_checksum(steps2[-1]))  # Should be 1928

In [6]:
actual_disk = create_initial_disk(actual_input.strip())
actual_steps = compact_disk(actual_disk)
print("\nActual:")
print("Initial:", ''.join(str(x) for x in actual_steps[0]))
print("Final:", ''.join(str(x) for x in actual_steps[-1]))
print("Checksum:", calculate_checksum(actual_steps[-1]))


Actual:
Initial: 00.11111111......22222222..3.........444445555555.66666..77777888..99999999.........10101010.111111111111......1212121212121212...131313131313131313...14141414.......151515151515151515..161616161616.....171717171717.181818181818181919191919191919192020202020.2121....222222222222222222..232323......242424.2525252525.......26........272727272727...282828282828282828........2929.30.......313131313131......32323232333333333333.........34343434343434....353535353535353536........3737373737373737.........3838383838.........39393939394040404040404040.....41414141....4242424242.....43......4444.454545....46464646464646..4747.........48484848484848.494949.........50505050505050.........51515252525252....5353..54545454.555555555555555555.......5656565656.5757575757.........5858........595959595959.........6060606060..6161616161...6262626262626262....6363636363......646464646464646464..656565....6666666666........67676767.....6868686868686868.....69696969...707070707070.........

In [7]:
def parse_disk_map(disk_map):
    """Convert disk map string to list of alternating file/space lengths."""
    return [int(c) for c in disk_map]

def create_initial_disk(disk_map):
    """Create initial disk state with file IDs and free spaces."""
    lengths = parse_disk_map(disk_map)
    disk = []
    file_id = 0
    
    for i in range(0, len(lengths), 2):
        # Add file blocks
        file_length = lengths[i]
        disk.extend([file_id] * file_length)
        file_id += 1
        
        # Add free space blocks if not at end
        if i + 1 < len(lengths):
            space_length = lengths[i + 1]
            disk.extend(['.'] * space_length)
    
    return disk

def get_file_info(disk):
    """Get information about each file's location and size."""
    file_info = {}  # {file_id: (start_pos, length)}
    current_id = None
    start_pos = None
    length = 0
    
    for i, block in enumerate(disk):
        if block != '.':
            if block != current_id:
                if current_id is not None:
                    file_info[current_id] = (start_pos, length)
                current_id = block
                start_pos = i
                length = 1
            else:
                length += 1
    
    # Don't forget the last file
    if current_id is not None:
        file_info[current_id] = (start_pos, length)
        
    return file_info

def find_leftmost_fitting_space(disk, required_length, start_after=0):
    """Find the leftmost span of free space that can fit the required length."""
    current_length = 0
    start_pos = None
    
    for i in range(start_after):
        if disk[i] != '.':
            current_length = 0
            start_pos = None
    
    for i in range(max(start_after, 0), len(disk)):
        if disk[i] == '.':
            if start_pos is None:
                start_pos = i
            current_length += 1
            if current_length >= required_length:
                return start_pos
        else:
            current_length = 0
            start_pos = None
    
    return None

def move_file(disk, from_pos, to_pos, length):
    """Move a file from one position to another."""
    file_id = disk[from_pos]
    # Clear original location
    for i in range(length):
        disk[from_pos + i] = '.'
    # Place at new location
    for i in range(length):
        disk[to_pos + i] = file_id

def compact_disk_whole_files(disk):
    """Compact the disk by moving whole files in order of decreasing file ID."""
    steps = [disk.copy()]
    
    # Get information about all files
    file_info = get_file_info(disk)
    
    # Process files in order of decreasing ID
    for file_id in sorted(file_info.keys(), reverse=True):
        start_pos, length = file_info[file_id]
        
        # Find leftmost fitting space before this file
        new_pos = find_leftmost_fitting_space(disk, length, 0)
        
        # If we found a suitable position and it's to the left of the current position
        if new_pos is not None and new_pos < start_pos:
            move_file(disk, start_pos, new_pos, length)
            steps.append(disk.copy())
            
            # Update file info for remaining files
            file_info = get_file_info(disk)
    
    return steps

def calculate_checksum(disk):
    """Calculate filesystem checksum."""
    checksum = 0
    for pos, block in enumerate(disk):
        if block != '.':
            checksum += pos * block
    return checksum

def solve_disk_fragmenter_part2(disk_map):
    """Solve part 2 of the disk fragmenter puzzle."""
    # Create initial disk state
    disk = create_initial_disk(disk_map)
    
    # Compact the disk using whole-file movement
    final_state = compact_disk_whole_files(disk)[-1]
    
    # Calculate checksum
    return calculate_checksum(final_state)

# Test with examples
example2 = "2333133121414131402"

# Print initial and final states for example
disk2 = create_initial_disk(example2)
steps2 = compact_disk_whole_files(disk2)
print("\nExample:")
print("Initial:", ''.join(str(x) for x in steps2[0]))
print("Final:", ''.join(str(x) for x in steps2[-1]))
print("Checksum:", calculate_checksum(steps2[-1]))  # Should be 2858


Example:
Initial: 00...111...2...333.44.5555.6666.777.888899
Final: 00992111777.44.333....5555.6666.....8888..
Checksum: 2858


In [8]:
solve_disk_fragmenter_part2(actual_input.strip())

6360363199987